In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SAK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [ ]:
list(zip(longforms, counts))

In [ ]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

In [ ]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [5]:
grounding_map, names, pos_labels = ({'staphylokinase': 'UP:C3VIQK'},
 {'UP:C3VIQK': 'Staphylokinase'},
 ['UP:C3VIQK'])

In [6]:
excluded_longforms = []

In [7]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [8]:
additional_entities = {'HGNC:11397': ['PLK4', ['SAK', 'Sak']]}

In [9]:
unambiguous_agent_texts = {'HGNC:11397': ['PLK4', ['PLK4', 'polo like kinase 4']]}

In [10]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [11]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [12]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [13]:
intersection1

[('HGNC:11397', 'HGNC:11397', 0)]

In [14]:
intersection2

[('UP:C3VIQK', 'HGNC:11397', 0)]

In [15]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [16]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-12-01 04:55:24] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-12-01 04:55:28] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9862068965517242 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'UP:C3VIQK': 41, 'HGNC:11397': 105},
 'f1': {'mean': 0.986207, 'std': 0.027586},
 'precision': {'mean': 0.986207, 'std': 0.027586},
 'recall': {'mean': 0.986207, 'std': 0.027586},
 'HGNC:11397': {'f1': {'mean': 0.990909, 'std': 0.018182},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.982609, 'std': 0.034783}},
 'UP:C3VIQK': {'f1': {'mean': 0.971429, 'std': 0.057143},
  'pr': {'mean': 0.95, 'std': 0.1},
  'rc': {'mean': 1.0, 'std': 0.0}}}

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
classifier.other_metadata = {'unambiguous_agent_texts': unambiguous_agent_texts}

In [21]:
disamb.dump(model_name, results_path)

In [22]:
print(disamb.info())

Disambiguation model for SAK

Produces the disambiguations:
	PLK4*	HGNC:11397
	Staphylokinase*	UP:C3VIQK

Class level metrics:
--------------------
Grounding     	Count	F1     
          PLK4*	105	0.99091
Staphylokinase*	 41	0.97143

Global Metrics:
-----------------
	F1 score:	0.98621
	Precision:	0.98621
	Recall:		0.98621

* Positive labels
See Docstring for explanation



In [23]:
model_to_s3(disamb)

In [1]:
from adeft.disambiguate import load_disambiguator

In [2]:
ad = load_disambiguator('SAK')

In [3]:
ad.labels

{'UP:P68802'}

In [4]:
ad.pos_labels

['UP:P68802']

In [5]:
ad.info()

'Disambiguation model for SAK\n\nProduces the disambiguations:\n\tStaphylokinase*\tUP:P68802\n\nClass level metrics:\n--------------------\nGrounding     \tCount\tF1     \n    Ungrounded\t77\t       \nStaphylokinase*\t39\t       \n\nGlobal Metrics:\n-----------------\n\tF1 score:\t0.98667\n\tPrecision:\t1.0\n\tRecall:\t\t0.975\n\n* Positive labels\nSee Docstring for explanation\n'

In [6]:
print(ad.info())

Disambiguation model for SAK

Produces the disambiguations:
	Staphylokinase*	UP:P68802

Class level metrics:
--------------------
Grounding     	Count	F1     
    Ungrounded	77	       
Staphylokinase*	39	       

Global Metrics:
-----------------
	F1 score:	0.98667
	Precision:	1.0
	Recall:		0.975

* Positive labels
See Docstring for explanation



In [7]:
ad = load_disambiguator('FP')

In [8]:
ad.labels

{'CHEBI:CHEBI:24018',
 'CHEBI:CHEBI:26347',
 'CHEBI:CHEBI:31441',
 'CHEBI:CHEBI:34922',
 'CHEBI:CHEBI:42699',
 'CHEBI:CHEBI:47344',
 'CHEBI:CHEBI:50145',
 'CHEBI:CHEBI:5063',
 'CHEBI:CHEBI:5086',
 'CHEBI:CHEBI:5123',
 'CHEBI:CHEBI:5130',
 'CHEBI:CHEBI:5134',
 'GO:GO:0020016',
 'HGNC:9600',
 'MESH:D000070037',
 'MESH:D000273',
 'MESH:D005189',
 'MESH:D005193',
 'MESH:D005454',
 'MESH:D005498',
 'MESH:D005587',
 'MESH:D008164',
 'MESH:D009240',
 'MESH:D010821',
 'MESH:D011429',
 'MESH:D012596',
 'MESH:D014760',
 'MESH:D052638',
 'MESH:D059247',
 'NCIT:C32623',
 'PUBCHEM:141643',
 'field_potential',
 'floor_plate',
 'fluid_percussion',
 'ungrounded'}

In [9]:
ad.grounding_dict

{'FP': {'fluticasone propionate': 'CHEBI:CHEBI:31441',
  'fluorescence polarization': 'MESH:D005454',
  'fusion peptide': 'MESH:D014760',
  'false positive': 'MESH:D005189',
  'fluorescence protein': 'MESH:D008164',
  'floor plate': 'floor_plate',
  'flavopiridol': 'CHEBI:CHEBI:47344',
  'fusion protein': 'MESH:D014760',
  'family planning': 'MESH:D005193',
  'field potential': 'field_potential',
  'flurbiprofen': 'CHEBI:CHEBI:5130',
  'flagellar pocket': 'GO:GO:0020016',
  'foot processes': 'NCIT:C32623',
  'first progression': 'ungrounded',
  'fat pad': 'MESH:D000273',
  'fluid percussion': 'fluid_percussion',
  'fertility preservation': 'MESH:D059247',
  'prostaglandin f2α receptor': 'HGNC:9600',
  'cp fine': 'MESH:D052638',
  'follicular phase': 'MESH:D005498',
  'fabry perot': 'ungrounded',
  'formyl peptide': 'MESH:D009240',
  'foreperiod': 'ungrounded',
  'fiber protein': 'MESH:D012596',
  'fluoropyrimidine': 'PUBCHEM:141643',
  'fat percentage': 'ungrounded',
  'fluorophosphona

In [10]:
print(ad.info())

Disambiguation model for FP, and FPs

Produces the disambiguations:
	2-Fluoropyrimidine	PUBCHEM:141643
	Adipose Tissue	MESH:D000273
	Faecalibacterium prausnitzii	MESH:D000070037
	False Positive Reactions*	MESH:D005189
	Family Planning Services*	MESH:D005193
	Fertility Preservation	MESH:D059247
	Fluorescence Polarization*	MESH:D005454
	Follicular Phase	MESH:D005498
	Foot Process*	NCIT:C32623
	Fowlpox virus	MESH:D005587
	Luminescent Proteins*	MESH:D008164
	N-Formylmethionine Leucyl-Phenylalanine	MESH:D009240
	PTGFR	HGNC:9600
	Particulate Matter	MESH:D052638
	Physicians, Family	MESH:D010821
	Propolis	MESH:D011429
	Scleroproteins	MESH:D012596
	Viral Fusion Proteins*	MESH:D014760
	alvocidib*	CHEBI:CHEBI:47344
	ciliary pocket*	GO:GO:0020016
	farnesyl phosphate	CHEBI:CHEBI:24018
	fenpropimorph	CHEBI:CHEBI:50145
	field_potential	field_potential
	fipronil	CHEBI:CHEBI:5063
	flavoprotein	CHEBI:CHEBI:5086
	floor_plate	floor_plate
	fluid_percussion	fluid_percussion
	fluoridophosphate	CHEBI:CHEBI:42

In [58]:
disamb

In [60]:
disamb.pos_labels

['UP:C3VIQK', 'HGNC:11397']

In [61]:
disamb.labels

{'UP:C3VIQK'}

In [62]:
disamb.pos_labels

['UP:C3VIQK', 'HGNC:11397']

In [63]:
disamb.classifier.other_metadata

{'unambiguous_agent_texts': {'HGNC:11397': ['PLK4',
   ['PLK4', 'polo like kinase 4']]}}

In [1]:
from adeft import __version__

In [2]:
__version__

'0.10.0'